# 합성 테스트 데이터셋 생성

- 합성테스트(Synthetic data)

RAGAS를 쓰는이유? 

RAGAS에는 특별한 알고리즘이 있다. 문서가 있으면, 문서안에서 사용자가 잠정적으로 낼수있는 질문과 답변을 만들어낸다. seed data를 가지고 좋은질문과 나쁜질문을 판별을하고 그에대해서 답변한다. 여기서 qa샘플로 평가를한다.

 문서를 주면 문서를 기반으로 질문과 답변을 만들어서(qa샘플을 만들어서) 우리의 질문과 얼마나 유사도가 높은지 평가를한다.

 **ragas의 요청사항**
 각 문서 객체에는 `metadata`를 통해 액세스할 수 있는 문서에 대한 추가 정보를 저장하는데 사용할 수 있는 메타데이터 사전이 포함되어있다.

 메타데이터 사전에는 `filename`이라는 키가 포함되어야 한다. 이 키는 Test datasets 생성 프로세스에 활용될 것 이므로, 메타데이터의 filename속성은 동일한 무서에 속한 청크를 식별하는데 도움이 된다.

 - RAGAS Dataset은 검수가 필수적이다


In [ ]:
# pip install ragas

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.document_loaders import PDFPlumberLoader

loader = PDFPlumberLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

docs = loader.load()

# 목차, 끝페이지 제외. 이곳에서 질문을 만들면 인되기 때문. 이 문서를 ragas에 전달한다.
docs = docs[3:-1]

len(docs)

19

In [3]:
docs[0].metadata

{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'page': 3,
 'total_pages': 23,
 'Author': 'dj',
 'Creator': 'Hwp 2018 10.0.0.13462',
 'Producer': 'Hancom PDF 1.3.0.542',
 'CreationDate': "D:20231208132838+09'00'",
 'ModDate': "D:20231208132838+09'00'",
 'PDFVersion': '1.4'}

In [4]:
# 메타데이터에 `filename`을 추가해주는 과정. filename을 추가하는데 source라는 metadata를 넣어준다.
for doc in docs:
    doc.metadata["filename"] = doc.metadata["source"]

In [5]:
docs[0].metadata

{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'file_path': 'data/SPRI_AI_Brief_2023년12월호_F.pdf',
 'page': 3,
 'total_pages': 23,
 'Author': 'dj',
 'Creator': 'Hwp 2018 10.0.0.13462',
 'Producer': 'Hancom PDF 1.3.0.542',
 'CreationDate': "D:20231208132838+09'00'",
 'ModDate': "D:20231208132838+09'00'",
 'PDFVersion': '1.4',
 'filename': 'data/SPRI_AI_Brief_2023년12월호_F.pdf'}

# 데이터셋 생성

In [7]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context, conditional
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset.extractor import KeyphraseExtractor
from ragas.testset.docstore import InMemoryDocumentStore

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 데이터셋 생성기 : 이놈은 좋은걸 써주는게 좋음. local llm을 쓰면 질문 답변이 좋지않을수 있다.
generator_llm = ChatOpenAI(model="gpt-4o-mini")
# 데이터셋 비평기 : generator llm에서 질문을 만들어내면, 이 질문이 적합한지를 평가를한다.
critic_llm = ChatOpenAI(model="gpt-4o-mini")
# 문서 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

DocumentStore를 초기화 합니다. 사용자 정의 LLM과 임베딩을 사용합니다.

In [8]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

langchain_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))

# 주요구문 추출기. 문서들이 있으면 중요한 단락을 추출한다.
keyphrase_extractor = KeyphraseExtractor(llm=langchain_llm)

ragas_embeddings = LangchainEmbeddingsWrapper(embeddings)

# InmemoryDocumentStore를 초기화한다.
# 이는 문서를 메모리에저장하고 관리하는 저장소다.
docstore = InMemoryDocumentStore(
    splitter=splitter,
    embeddings=ragas_embeddings,
    extractor=keyphrase_extractor
)

TestSet 생성

In [9]:
generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    ragas_embeddings,
    docstore=docstore
)

**질문의 유형별 분포**
- simple: 간단한 질문
- reasoning: 추론이 필요한 질문
- multi_context: 여러 맥락을 고려해야 하는 질문
- conditional: 조건부 질문

In [10]:
# 질문 유형별 분포 결정
# simple: 간단한 질문, reasoning: 추론이 필요한 질문, multi_context: 여러 맥락을 고려해야 하는 질문, conditional: 조건부 질문
distributions = {simple: 0.4, reasoning: 0.2, multi_context: 0.2, conditional: 0.2}

- documents: 문서 데이터
- test_size: 생성할 질문의 수
- distributions: 질문 유형별 분포
- with_debugging_logs: 디버깅 로그 출력 여부

In [11]:
# 테스트셋 생성
# docs: 문서 데이터, 10: 생성할 질문의 수, distributions: 질문 유형별 분포, with_debugging_logs: 디버깅 로그 출력 여부
testset = generator.generate_with_langchain_docs(
    documents=docs, test_size=10, distributions=distributions, with_debugging_logs=True
)

embedding nodes:   0%|          | 0/74 [00:00<?, ?it/s]

Generating:   0%|          | 0/10 [00:00<?, ?it/s]

[ragas.testset.filters.DEBUG] context scoring: {'clarity': 2, 'depth': 3, 'structure': 2, 'relevance': 3, 'score': 2.5}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['AGI', 'Artificial Intelligence', 'Operationalizing Progress', 'Levels of AGI', 'Good Old Fashioned Artificial Intelligence (GOFAI)']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 2, 'depth': 3, 'structure': 2, 'relevance': 3, 'score': 2.5}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['AGI', 'Artificial Intelligence', 'Operationalizing Progress', 'Levels of AGI', 'Good Old Fashioned Artificial Intelligence (GOFAI)']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2, 'structure': 2, 'relevance': 3, 'score': 2.0}
[ragas.testset.evolutions.DEBUG] keyphrases in merged node: ['AI Safety Summit', 'AI governance', 'AI risk management', 'AI policy recommendations', 'AI development guidelines']
[ragas.testset.filters.DEBUG] context scoring: {'clarity': 1, 'depth': 2

In [12]:
# 생성된 테스트셋을 pandas DataFrame으로 변환
test_df = testset.to_pandas()
test_df

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What role does content moderation play in the ...,"[£유튜브, 특정인을 모방한 AI 생성 콘텐츠에 대한 삭제 요청에도 대응 계획\nn...",Content moderation plays a crucial role in AI ...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
1,What are the key aspects of the AI Development...,"[SPRi AI Brief |\n2023-12월호\nG7, 히로시마 AI 프로세스를...",The key aspects of the AI Development Guidelin...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
2,What was the purpose of the AI Safety Summit h...,[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n영국 AI 안전...,The purpose of the AI Safety Summit held in No...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
3,What does 'Operationalizing Progress' refer to...,"[구글 딥마인드, 범용 AI 모델의 기능과 동작에 대한 분류 체계 발표\nKEY C...",The term 'Operationalizing Progress' in the co...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
4,What improvements does Tongyi Qianwen 2.0 have...,[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n알리바바 클라우...,Tongyi Qianwen 2.0 has improvements over the p...,reasoning,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
5,What features of Claude LLM have advanced NLP ...,[저해상도 이미지의 고해상도 전환도 지원\nn IT 전문지 테크리퍼블릭(TechRe...,The answer to given question is not present in...,reasoning,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
6,How does data integrity support transparency i...,"[SPRi AI Brief |\n2023-12월호\n코히어, 데이터 투명성 확보를 ...",The answer to given question is not present in...,multi_context,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
7,What key AI events in 2024 will showcase the e...,[Ⅱ\n. 주요 행사 일정\n행사명 행사 주요 개요\n- 미국 소비자기술 협회(CT...,The key AI events in 2024 that will showcase t...,multi_context,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
8,What challenges come up in the EU's AI Act if ...,[∙ 3개국은 기반모델 전반에 대한 규제가 기술 중립적이고 위험 기반의 AI 규제 ...,The challenges that arise in the EU's AI Act i...,conditional,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
9,What factors matter for ethical AI innovation?,"[£유튜브, 특정인을 모방한 AI 생성 콘텐츠에 대한 삭제 요청에도 대응 계획\nn...",The context discusses various factors that mat...,conditional,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True


In [13]:
# DataFrame의 상위 5개 행 출력
test_df.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,What role does content moderation play in the ...,"[£유튜브, 특정인을 모방한 AI 생성 콘텐츠에 대한 삭제 요청에도 대응 계획\nn...",Content moderation plays a crucial role in AI ...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
1,What are the key aspects of the AI Development...,"[SPRi AI Brief |\n2023-12월호\nG7, 히로시마 AI 프로세스를...",The key aspects of the AI Development Guidelin...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
2,What was the purpose of the AI Safety Summit h...,[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n영국 AI 안전...,The purpose of the AI Safety Summit held in No...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
3,What does 'Operationalizing Progress' refer to...,"[구글 딥마인드, 범용 AI 모델의 기능과 동작에 대한 분류 체계 발표\nKEY C...",The term 'Operationalizing Progress' in the co...,simple,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True
4,What improvements does Tongyi Qianwen 2.0 have...,[1. 정책/법제 2. 기업/산업 3. 기술/연구 4. 인력/교육\n알리바바 클라우...,Tongyi Qianwen 2.0 has improvements over the p...,reasoning,[{'source': 'data/SPRI_AI_Brief_2023년12월호_F.pd...,True


In [14]:
# DataFrame을 CSV 파일로 저장
test_df.to_csv("data/ragas_synthetic_dataset.csv", index=False)